# データセットの操作

前のラボでは、*データストア*を使用して、クラウドベースのデータ アクセスを一元化しました。  このラボでは、*データセット*を詳しく調べ、さらなる抽象化を行い、実験やトレーニングのために特定のデータを簡単に操作できるようにします。

## ワークスペースに接続する

まず、Azure ML SDK を使用してワークスペースに接続する必要があります。

> **注**: 前回の演習を完了してから Azure サブスクリプションとの認証済みセッションの有効期限が切れている場合は、再認証を求めるメッセージが表示されます。

In [ ]:
import azureml.core
from azureml.core import Workspace

# 保存した構成ファイルからワークスペースを読み込む
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## データを準備する

前のラボでは、データストアを作成しました。データセットは、通常、データストアのデータに基づいています (必ずしもそうではありません)。

前のラボを完了していない場合は、次のコードを実行して、ワークスペースの既定のデータストアに 2 つのローカル CSV ファイルをアップロードします ( 前の演習を完了*した*場合は、同じファイルが上書きされます)。

In [ ]:
ws.get_default_datastore().upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'], # 糖尿病 CSV ファイルを /data にアップロードする
                       target_path='diabetes-data/', # データストアのフォルダー パスに入れる
                       overwrite=True, # 同じ名前の既存のファイルを置き換える
                       show_progress=True)

## 表形式データセットを作成する

データセットは、特定のデータソースをカプセル化するオブジェクトです。データストアにアップロードした糖尿病データからデータセットを作成し、最初の 20 件のレコードを表示してみましょう。この場合、データは CSV ファイル内の構造化された形式であるため、*表形式*のデータセットを使用します。

In [ ]:
from azureml.core import Dataset

# 既定のデータストアを取得する
default_ds = ws.get_default_datastore()

# データストア上のパスから表形式のデータセットを作成する (しばらく時間がかかる場合があります)
tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

# 最初の 20 行を Pandas データフレームとして表示する
tab_data_set.take(20).to_pandas_dataframe()

上のコードでわかるように、表形式のデータセットを Pandas データフレームに変換するのは簡単で、一般的な Python の手法を使用してデータを操作できます。

## ファイル データセットを作成する

作成したデータセットは、データセット定義に含まれる構造化ファイル内のすべてのデータを含むデータフレームとして読み取ることができる*表形式*のデータセットです。これは表形式のデータに適していますが、機械学習のシナリオによっては、非構造化データの操作が必要となる場合があります。または、単に自分のコード内のファイルからデータの読み取り処理を行うことが必要となる場合もあります。これを実現するには、*ファイル* データセットを使用して、ファイルのデータを読み取るために使用できる仮想マウント ポイント内のファイル パスのリストを作成します。

In [ ]:
# データストア上のパスからファイル データセットを作成する (しばらく時間がかかる場合があります)
file_data_set = Dataset.File.from_files(path=(default_ds, 'diabetes-data/*.csv'))

# データセット内のファイルを取得する
for file_path in file_data_set.to_path():
    print(file_path)

## データセットを登録する

これで、糖尿病データを参照するデータセットを作成したので、それらを登録して、ワークスペースで実行されている実験に簡単にアクセスできるようにすることができます。

表形式のデータセットを**糖尿病データセット**、ファイル データセットを**糖尿病ファイル**として登録します。

In [ ]:
# 表形式のデータセットを登録する
try:
    tab_data_set = tab_data_set.register(workspace=ws, 
                               name='diabetes dataset',
                               description='diabetes data',
                               tags = {'format':'CSV'},
                               create_new_version=True)
except Exception as ex:
    print(ex)

# ファイル データセットを登録する
try:
    file_data_set = file_data_set.register(workspace=ws, 
                               name='diabetes file dataset',
                               description='diabetes files',
                               tags = {'format':'CSV'},
                               create_new_version=True)
except Exception as ex:
    print(ex)

print('Datasets registered')

[Azure ML Studio](https://ml.azure.com) で、ワークスペースの**データセット** ページでデータセットを表示および管理できます。ワークスペース オブジェクトからもデータセットのリストを取得します。

In [ ]:
print("Datasets:")
for dataset_name in list(ws.datasets.keys()):
    dataset = Dataset.get_by_name(ws, dataset_name)
    print("\t", dataset.name, 'version', dataset.version)

ラボ 2A および 2B を完了すると、ビジュアル デザイナー ツールを使用して作成された変換が、登録されたデータセットに含まれていることがわかります。また、前の演習で *Studio* インターフェイスを使用して作成したデータセットと同じ名前で**糖尿病データセット**を登録する場合は、データセットの新しい*バージョン*を作成していることがわかります。データセットをバージョン管理できるため、以前の定義に依存する既存の実験やパイプラインを壊すことなくデータセットを再定義できます。既定では、名前付きデータセットの最新バージョンが返されますが、次のようにバージョン番号を指定することで、特定のバージョンのデータセットを取得できます。

```python
dataset_v1 = Dataset.get_by_name(ws, 'diabetes dataset', version = 1)
```


## 表形式データセットからモデルをトレーニングする

データセットができたので、そこからモデルのトレーニングを開始する準備が整いました。データセットは、スクリプトの実行に使用される Estimator で、*入力*としてスクリプトに渡すことができます。

次の 2 つのコード セルを実行して作成します。

1. **diabetes_training_from_tab_dataset** という名前のフォルダー
2. *入力*として渡される表形式のデータセットを使用して分類モデルをトレーニングするスクリプト。

In [ ]:
import os

# 実験ファイル用フォルダーを作成する
experiment_folder = 'diabetes_training_from_tab_dataset'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, 'folder created')

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# ライブラリをインポートする
import argparse
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# 正規化ハイパーパラメーターを設定する (スクリプトに引数として渡される)
parser = argparse.ArgumentParser()
parser.add_argument('--regularization', type=float, dest='reg_rate', default=0.01, help='regularization rate')
args = parser.parse_args()
reg = args.reg_rate

# 実験実行コンテキストを取得する
run = Run.get_context()

# 糖尿病データを読み込む (入力データセットとして渡される)
print("Loading Data...")
diabetes = run.input_datasets['diabetes'].to_pandas_dataframe()

# フィーチャーとラベルを分離する
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# データをトレーニング セットとテスト セットに分割する
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# ロジスティック回帰モデルをトレーニングする
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# 精度を計算する
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# AUC を計算する
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

os.makedirs('outputs', exist_ok=True)
# 出力フォルダーに保存されたファイルは、自動的に実験レコードにアップロードされます
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

これで、スクリプトを実行する Estimator を作成したので、スクリプトによって読み取られるトレーニング データセットの名前付き*入力*を定義できます。

> **注**: **Dataset** クラスは **azureml-dataprep** パッケージ (SDK と共にインストールされる) で定義されており、このパッケージには **Pandas** のオプションサポートが含まれています (**to_pandas_dataframe()** メソッドで使用されるため、トレーニング実験を実行する環境にこのパッケージを含める必要があります)。

In [ ]:
from azureml.train.sklearn import SKLearn
from azureml.core import Experiment
from azureml.widgets import RunDetails

# スクリプト パラメーターを設定する
script_params = {
    '--regularization': 0.1
}

# トレーニング データセットを取得する
diabetes_ds = ws.datasets.get("diabetes dataset")

# Estimator を作成する
estimator = SKLearn(source_directory=experiment_folder,
                    entry_script='diabetes_training.py',
                    script_params=script_params,
                    compute_target = 'local',
                    inputs=[diabetes_ds.as_named_input('diabetes')], # データセット オブジェクトを入力として渡す...
                    pip_packages=['azureml-dataprep[pandas]'] # ...したがって、データ準備パッケージが必要です
                   )

# 実験を作成する
experiment_name = 'diabetes-training'
experiment = Experiment(workspace = ws, name = experiment_name)

# 実験を実行する
run = experiment.submit(config=estimator)
# 実行中に実行の詳細を表示する
RunDetails(run).show()
run.wait_for_completion()

初めて実験を実行すると、Python 環境のセットアップに時間がかかる場合があります。以降の実行はより高速になります。

実験が完了したら、ウィジェットで、**azureml-logs/70_driver_log.txt** 出力ログと実行によって生成されたメトリックを表示します。

すべての実験と同様に、[Azure ML Studio](https://ml.azure.com) で実行された実験の詳細を表示したり、生成されたメトリックとファイルを取得するコードを書き込んだりできます。

In [ ]:
# 指標の記録を取得する
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

トレーニングしたモデルは、**outputs** フォルダーに **diabetes_model.pkl** ファイルとして保存されるので、登録することができます。

In [ ]:
from azureml.core import Model

run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'SKLearn Estimator (tabular dataset)'}, properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

## ファイル データセットからモデルをトレーニングする

*表形式*データセットでトレーニング データを使用してモデルをトレーニングする方法を見てきましたが、*ファイル* データセットについてはどうでしょうか。

ファイル データセットを使用する場合、スクリプトに渡されるデータセット入力は、ファイル パスを含むマウント ポイントを表します。これらのファイルからデータを読み取る方法は、ファイル内のデータの種類と、そのデータを使用して何を行うかによって異なります。糖尿病 CSV ファイルの場合、Python **glob** モジュールを使用して、データセットによって定義された仮想マウント ポイント内のファイルのリストを作成し、それらをすべて単一のデータフレームに連結された Pandas データフレームに読み込むことができます。

次の 2 つのコード セルを実行して作成します。

1. **diabetes_training_from_file_dataset** という名前のフォルダー
2. *入力*として渡されるファイル データセットを使用して分類モデルをトレーニングするスクリプト。

In [ ]:
import os

# 実験ファイル用フォルダーを作成する
experiment_folder = 'diabetes_training_from_file_dataset'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, 'folder created')

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# ライブラリをインポートする
import argparse
from azureml.core import Workspace, Dataset, Experiment, Run
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import glob

# 正規化ハイパーパラメーターを設定する (スクリプトに引数として渡される)
parser = argparse.ArgumentParser()
parser.add_argument('--regularization', type=float, dest='reg_rate', default=0.01, help='regularization rate')
args = parser.parse_args()
reg = args.reg_rate

# 実験実行コンテキストを取得する
run = Run.get_context()

# 糖尿病データセットを読み込む
print("Loading Data...")
data_path = run.input_datasets['diabetes'] # Estimator 入力からトレーニング データを取得する
all_files = glob.glob(data_path + "/*.csv")
diabetes = pd.concat((pd.read_csv(f) for f in all_files))

# フィーチャーとラベルを分離する
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# データをトレーニング セットとテスト セットに分割する
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# ロジスティック回帰モデルをトレーニングする
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# 精度を計算する
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# AUC を計算する
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

os.makedirs('outputs', exist_ok=True)
# 出力フォルダーに保存されたファイルは、自動的に実験レコードにアップロードされます
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

次に、データセットを Estimator に渡す方法を変更する必要があります。スクリプトがファイルを読み取ることができるマウント ポイントを定義する必要があります。通常、大量のデータの場合、**as_mount** メソッドを使用して、データセット ソースから直接ファイルをストリーミングしますが、ローカル コンピューティングで実行する場合 (この例のように) は、**as_download** オプションを使用して、データセット ファイルをローカル フォルダーにダウンロードする必要があります。

また、**Dataset** クラスは **azureml-dataprep** パッケージで定義されているため、実験環境に含める必要があります。

In [ ]:
from azureml.train.sklearn import SKLearn
from azureml.core import Experiment
from azureml.widgets import RunDetails

# スクリプト パラメーターを設定する
script_params = {
    '--regularization': 0.1
}

# トレーニング データセットを取得する
diabetes_ds = ws.datasets.get("diabetes file dataset")

# Estimator を作成する
estimator = SKLearn(source_directory=experiment_folder,
                    entry_script='diabetes_training.py',
                    script_params=script_params,
                    compute_target = 'local',
                    inputs=[diabetes_ds.as_named_input('diabetes').as_download(path_on_compute='diabetes_data')], # データセット オブジェクトを入力として渡す
                    pip_packages=['azureml-dataprep[pandas]'] # したがって、データ準備パッケージが必要です
                   )

# 実験を作成する
experiment_name = 'diabetes-training'
experiment = Experiment(workspace = ws, name = experiment_name)

# 実験を実行する
run = experiment.submit(config=estimator)
# 実行中に実行の詳細を表示する
RunDetails(run).show()
run.wait_for_completion()

実験が完了したら、ウィジェットで **azureml-logs/70_driver_log.txt** 出力ログを表示し、ファイル データセットが処理され、データ ファイルがダウンロードされたことを確認します。

すべての実験と同様に、[Azure ML Studio](https://ml.azure.com) で実行された実験の詳細を表示したり、生成されたメトリックとファイルを取得するコードを書き込んだりできます。

In [ ]:
# 指標の記録を取得する
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

もう一度、トレーニングしたモデルを登録してみましょう。

In [ ]:
from azureml.core import Model

run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'SKLearn Estimator (file dataset)'}, properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

> **詳細情報**: データセットを使用するトレーニングの詳細については、Azure ML ドキュメントの[データセットを使用するトレーニング](https://docs.microsoft.com/azure/machine-learning/how-to-train-with-datasets)を参照してください。